In [1]:
import os, sys
sys.path.append("./core")

import numpy as np
from importlib import reload

from gf2 import gf2, gf2matrix
# gf2matrix is not in gf2.py, so do not import it here

In [2]:
a = 1
b = 0

gf2.div(b,a)

0

In [3]:
random_mat = gf2matrix.random(3, 3, density=0.5)
print(random_mat)

[[0 1 0]
 [0 1 1]
 [0 1 0]]


In [4]:
r = gf2matrix.random(3, 3, density=0.5)
v = [1,0,1]

print("r1:",r)
print("v:",v)
print("r*v:", r.apply(v))

r1: [[1 0 1]
 [1 0 0]
 [0 0 1]]
v: [1, 0, 1]
r*v: [0 1 1]


In [5]:
import berlekamp_massey
reload(berlekamp_massey)

<module 'berlekamp_massey' from 'c:\\Users\\bagof\\OneDrive - Duke University\\Duke\\Extracurriculars\\ECC Research\\Wiedemann\\core\\berlekamp_massey.py'>

In [6]:
# Example sequence over GF(2)
# seq = [0, 0, 1, 1, 0, 1, 0]
seq = [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0]  # input sequence

# Run Berlekamp-Massey algorithm
lfsr_poly = berlekamp_massey.berlekamp_massey.find_minimal_polynomial(seq)

print("Input sequence:", seq)
print("Minimal polynomial (LFSR):", lfsr_poly)
# Should get [1, 1, 0, 1] for the example sequence --> C(x)=1 + x + x^3

Input sequence: [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0]
Minimal polynomial (LFSR): [1, 1, 1, 0, 0, 0, 1, 0]


In [7]:
# Berlekamp-Massey Algorithm over GF(2)

s = [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0]  # input sequence
# s = [0,0,1,1,0,1,0]
n = len(s)
C = [1]
B = [1]
L = 0
m = -1

for N in range(n):
    # Compute discrepancy d
    d = s[N]
    for i in range(1, L + 1):
        d += C[i] * s[N - i] if N - i >= 0 else 0
    if d == 1:
        temp = C.copy()
        # Pad B with zeros
        pad = [0] * (N - m)
        Bpad = pad + B
        # Extend C or Bpad to same length
        if len(C) < len(Bpad):
            C += [0] * (len(Bpad) - len(C))
        else:
            Bpad += [0] * (len(C) - len(Bpad))
        # Update C
        C = [c + b for c, b in zip(C, Bpad)]
        if L <= N // 2:
            L = N + 1 - L
            m = N
            B = temp
    # Optionally print progress
    # print(f'N={N}, L={L}, C={C}')

# Print the minimal polynomial
print('-----------------------------------------------------------------------')
print('Solution is:', end=' ')
# terms = []
# for j, coeff in enumerate(C):
#     if coeff == 1:
#         if j == 0:
#             terms.append('1')
#         elif j == 1:
#             terms.append('x')
#         else:
#             terms.append(f'x^{j}')
# print(' + '.join(terms))
print(C)


-----------------------------------------------------------------------
Solution is: [1, 1, 1, 2, 2, 2, 1]


In [8]:
MOD = 2
def fp(a, k):
    return pow(a, k, MOD)
def bm(a):
    n = len(a) - 1  
    ans_coef = []  
    lst = []  
    w = 0  
    delta = 0  
    for i in range(1, n + 1):
        tmp = 0
        for j in range(len(ans_coef)):
            if i - 1 - j >= 1:
                tmp = (tmp + a[i - 1 - j] * ans_coef[j]) % MOD
        discrepancy = (a[i] - tmp + MOD) % MOD
        if discrepancy == 0:
            continue  
        if w == 0:
            ans_coef = [0] * i  
            w = i
            delta = discrepancy
            continue
        now = list(ans_coef)
        mul = discrepancy * fp(delta, MOD - 2) % MOD
        needed_len = len(lst) + i - w
        if len(ans_coef) < needed_len:
            ans_coef.extend([0] * (needed_len - len(ans_coef)))
        if i - w - 1 >= 0:
            ans_coef[i - w - 1] = (ans_coef[i - w - 1] + mul) % MOD
        for j in range(len(lst)):
            idx = i - w + j
            if idx < len(ans_coef):  
                term_to_subtract = (mul * lst[j]) % MOD
                ans_coef[idx] = (ans_coef[idx] - term_to_subtract + MOD) % MOD
        if len(ans_coef) > len(now):  
            lst = now
            w = i
            delta = discrepancy
    return [(x + MOD) % MOD for x in ans_coef]
def calculate_term(m, coef, h):
    k = len(coef)
    if m < len(h):
        return (h[m] + MOD) % MOD
    if k == 0:  
        return 0
    p_coeffs = [0] * (k + 1)
    p_coeffs[0] = (MOD - 1) % MOD  
    for i in range(k):
        p_coeffs[i + 1] = coef[i]
    def poly_mul(a, b, degree_k, p_poly):
        res = [0] * (2 * degree_k)
        for i in range(degree_k):
            if a[i] == 0: continue
            for j in range(degree_k):
                res[i + j] = (res[i + j] + a[i] * b[j]) % MOD
        for i in range(2 * degree_k - 1, degree_k - 1, -1):
            if res[i] == 0: continue
            term = res[i]
            res[i] = 0  
            for j in range(degree_k + 1):
                idx = i - j
                if idx >= 0:
                    res[idx] = (res[idx] + term * p_poly[j]) % MOD
        return res[:degree_k]
    f = [0] * k
    g = [0] * k
    f[0] = 1  
    if k == 1:
        if k == 1:
            g[0] = p_coeffs[1]  
        else:
            g[1] = 1  
    else:  
        g[1] = 1  
    power = m
    while power > 0:
        if power & 1:
            f = poly_mul(f, g, k, p_coeffs)
        g = poly_mul(g, g, k, p_coeffs)
        power >>= 1
    final_ans = 0
    for i in range(k):
        if i + 1 < len(h):
            final_ans = (final_ans + h[i + 1] * f[i]) % MOD
    return (final_ans + MOD) % MOD
def solve():
    h_input = [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0]
    n=len(h_input)
    h = [0] + h_input
    ans_coef = bm(h)
    print(*((x + MOD) % MOD for x in ans_coef))
    # m=10
    # result = calculate_term(m, ans_coef, h)
    # print((result + MOD) % MOD)

solve()

1 1 0 0 0 1


### Comparing Berlekamp–Massey Implementations

The three implementations above differ in the following ways:

- **`berlekamp_massey.min_poly`**: A static GF(2) version returning a full connection polynomial `C(x) = c_0 + c_1 x + ... + c_L x^L`, with arithmetic optimized for XOR.
- **Manual GF(2) cell**: Uses explicit integer addition/multiplication mod 2 and index offsets (`m`, `w`) that may shift coefficients differently (off-by-one in shift conventions).
- **Generic `bm(a)` function**: Works over any prime modulus (here `MOD = 2`), uses inverses for discrepancy normalization, and may return coefficients without the leading `c_0 = 1`.

All three should satisfy the annihilation property: for a polynomial of degree `L`, the sequence convolution

```
\sum_{j=0}^L c_j * s[i - j] ≡ 0  (mod MOD)
```

for all `i ≥ L`. We’ll add a test below to verify each returned polynomial on the same input sequence.

In [9]:
# Validation: test annihilation property for each implementation

def test_annihilation(sequence, poly):
    L = len(poly) - 1
    # verify for all i >= L
    for i in range(L, len(sequence)):
        acc = 0
        for j, c in enumerate(poly):
            acc = (acc + c * sequence[i - j]) % MOD
        if acc != 0:
            return False
    return True

# Input sequence
s = np.random.randint(0, 2, 8).astype(list)  # Random sequence of 0s and 1s

# 1st implementation (min_poly)
poly1 = berlekamp_massey.berlekamp_massey.find_minimal_polynomial(s)
# 2nd manual cell C
# reuse C from the manual cell above (compute again if needed)
# poly2 = C
# 3rd generic bm(a)
poly3 = bm([0] + s)

print("poly1:", poly1, "valid:", test_annihilation(s, poly1))
#print("poly2:", poly2, "valid:", test_annihilation(s, poly2))
print("poly3:", poly3, "valid:", test_annihilation([0]+s, poly3))

poly1: [1, 0, 1, 1, 0, 0, 0] valid: True
poly3: [0, 1, 1] valid: False


In [10]:
from scalar_wiedemann import ScalarWiedemann

In [42]:
# Test the implementation
def test_scalar_wiedemann():
    """Test the scalar Wiedemann implementation with a known matrix and solution"""
    # Create a matrix and a solution
    n = 5
    A_dense = np.array([
        [1, 1, 0, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1],
        [0, 0, 1, 0, 1],
        [1, 0, 0, 0, 1]
    ], dtype=np.int8)
    
    A = gf2matrix.from_dense(A_dense)
    
    # Choose a true solution
    x_true = np.array([1, 0, 1, 0, 1], dtype=np.int8)
    
    # Calculate the right-hand side b = Ax
    b = A.apply(x_true)
    
    print("Matrix A:")
    print(A)
    print("\nTrue solution x_true:")
    print(x_true)
    print("\nRight-hand side b = Ax:")
    print(b)
    
    # Solve using the Scalar Wiedemann Algorithm
    x_solution = ScalarWiedemann.solve(A)
    
    # Check the solution
    if x_solution is not None:
        print("\nComputed solution x_solution:")
        print(x_solution)
        
        # Verify: A * x_solution should equal b
        b_verify = A.apply(x_solution)
        print("\nFinal verification: A * x_solution =")
        print(b_verify)
        print("Matches b:", np.array_equal(b, b_verify))
        print("Matches x_true:", np.array_equal(x_true, x_solution))
    else:
        print("\nNo solution was found.")

# Run the test
test_scalar_wiedemann()

# Also demonstrate with a random sparse matrix
def test_random_matrix():
    """Test the scalar Wiedemann implementation with a random matrix"""
    print("\n\nTesting with a random sparse matrix:")
    
    # Create a random sparse matrix
    n = 8
    A = gf2matrix.random(n, density=0.3)
    
    print("Random sparse matrix A:")
    print(A)
    
    x_solution = ScalarWiedemann.solve(A)
    if x_solution is not None:
        print("\nComputed solution x_solution:")
        print(x_solution)
        
        # Verify: A * x_solution should equal 0
        b = A.apply(x_solution)
        print("\nRight-hand side Ax=0:")
        print(b)
        print("Matches zero vector:", np.array_equal(b, np.zeros(n, dtype=np.int8)))
    else:
        print("\nNo solution was found.")

# Run the test with a random matrix
test_random_matrix()

Matrix A:
[[1 1 0 0 1]
 [0 0 0 0 0]
 [0 0 0 1 1]
 [0 0 1 0 1]
 [1 0 0 0 1]]

True solution x_true:
[1 0 1 0 1]

Right-hand side b = Ax:
[0 0 1 0 0]

Attempt 1 with a different random vector u:
Minimal polynomial: [1, 1, 0, 0]
Minimal polynomial does not annihilate S, trying another y vector.

Attempt 2 with a different random vector u:
Minimal polynomial: [1, 1, 0]
Minimal polynomial does not annihilate S, trying another y vector.

Attempt 3 with a different random vector u:
Minimal polynomial: [1, 0, 1, 0, 0, 0]
Minimal polynomial does not annihilate S, trying another y vector.

Attempt 4 with a different random vector u:
Minimal polynomial: [1, 0, 1, 0, 0]
Minimal polynomial does not annihilate S, trying another y vector.

Attempt 5 with a different random vector u:
Minimal polynomial: [1, 0, 0, 0]
Minimal polynomial does not annihilate S, trying another y vector.

No solution was found.


Testing with a random sparse matrix:
Random sparse matrix A:
[[0 0 0 1 0 1 0 1]
 [0 1 1 0 0 0 1